### 소인수 분해(Prime Factorization)
- 1보다 큰 정수 n을 소수의 곱으로 표현한 것
$$n=p_1^{k_1}\times p_2^{k_2}\times \cdots \times p_m^{k_m}$$

#### <font color="red">산술의 기본정리(Fundamental Theorem of Arithmetic)</font>
- 1보다 큰 정수 n은 모두 유일한 소수의 곱으로 표현할 수 있다.
- <font color="red">유일한 인수분해 정리(Unique Factorization Theorem)</font>
    - 1보다 큰 정수 n을 인수분해하는 방법은 유일하다.
$$p_1<p_2<\cdots<p_m$$

#### 수학적 정의들
- <font color="red">소수</font>(prime): 1과 자신만을 약수(인수)로 가지는 1보다 큰 정수
- 합성수(composite): 1보다 큰 정수 중에서 소수가 아닌 수
- <font color="red">소인수</font>(prime factor): 주어진 정수의 소수인 인수(약수)
- <font color="red">인수</font>(factor): 주어진 정수의 약수(divisor)
    - 약수(divisor): 정수 n, m, k에 대하여 n = km (k != 0)일 때, k를 n의 약수라 한다.
- <font color="red">서로소</font>(relative prime): 최대 공약수가 1인 두 정수를 서로소라 한다.
    - 둘 다 0이 아닌 두 정수 n, m에 대하여 gcd(n,m) = 1이면, n과 m은 서로소

#### 첫 번째 소인수 분해 알고리즘
- 입력: 1보다 큰 정수 n
- 출력: 소인수와 차수(order)의 딕셔너리
    - {p1:k1, p2:k2, ..., pm:km}
- 입출력 예시
    - 입력: n = 360
    - 출력: {2:3, 3:2, 5:1}
- 알고리즘
    - 2부터 sqrt(n)까지의 정수 i에 대해서 순차적으로 반복
        - i가 n의 약수라면 딕셔너리에 추가 (차수는 1로 초기화, 딕셔너리에 이미 해당 약수가 존재하면 차수를 1 증가)
        - n은 n을 i로 나눈 몫으로 변경
        - i가 n의 약수가 아닐 때까지 반복

In [128]:
def add_factor(factors, f):
    if f not in factors:
        factors[f] = 1
    else:
        factors[f] += 1


def factorize_1(n):
    prime_factors = {}
    
    for i in range(2, int(n**0.5)+1):
        while n % i == 0:
            add_factor(prime_factors, i)
            n //= i
            
    if n > 1:
        add_factor(prime_factors, n)
        
    return prime_factors

n = 360
print(f"{n} : {factorize_1(n)}")

360 : {2: 3, 3: 2, 5: 1}


- 메르센 소수(첫 번째 소인수 분해 알고리즘으로 구현)

In [132]:
import time


num_of_prime = 0
start = time.time()
for i in range(2, 50):
    n = 2**i - 1
    factorization_1 = factorize_1(n)
    is_prime = "*" if len(factorization_1) == 1 else ""
    if is_prime:
        num_of_prime += 1
    # print(f"{i}, {n} : {factorization_1} {is_prime}")
end = time.time()
print(f"# of Prime : {num_of_prime}")
print(f"실행 시간 : {end - start:.4f}s")

# of Prime : 8
실행 시간 : 6.3390s


#### 성능 개선을 위한 아이디어

- 소인수는 소수이기 때문에, 우선 1부터 |sqrt(n)|까지의 모든 소수를 먼저 찾는다.
    - primes = find_primes(sqrt(n))
- primes 에 있는 소수들로만 소인수 분해를 진행한다.

#### 소수 판별하기

- 2부터 sqrt(n)까지 i에 대하여 반복하여 실행
    - i가 n의 약수라면 n은 소수가 아님
    - 위 과정을 끝까지 반복했을 때, 모든 i에 대하여 약수가 아니면 소수

In [55]:
def is_prime(n):
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
    return True


def find_primes(n):
    primes = []
    for i in range(2, n + 1):
        if is_prime(i):
            primes.append(i)
    return primes

n = 2**21 - 1
%time print(f"{n} : {len(find_primes(n))}")

2097151 : 155611
CPU times: user 11.3 s, sys: 75.2 ms, total: 11.3 s
Wall time: 11.5 s


- 에라토스테네스의 체(Eratosthenes' Sieve)
    - 체(Sieve): 체(도구)는 가루를 곱체 치거나 액체를 거르는 데 쓰는 기구이다.
    - 1부터 n까지의 모든 소수를 효율적으로 찾는 알고리즘
    - 시간복잡도가 O(n * log(log(n))) 이다.
        - O(n * log(log(n))) ≒ O(n): n에 대해서는 거의 선형 시간이 소요된다.
        - 하지만, n의 비트 수 b에 대해서는 여전히 지수 시간인 O(2^b)이다.

In [46]:
# 에라토스테네스의 체(Eratosthenes' Sieve) 구현
# import math


def sieve(n):
    flags = [0, 0] + [1] * (n-1)
    # sqrtn = math.floor(math.sqrt(n))
    for i in range(2, int(n**0.5) + 1):
        if flags[i] == 1:
            for j in range(i*i, n+1, i):  # 2 x i -> i x i 로 변경
                flags[j] = 0
    primes = []
    for i in range(len(flags)):
        if flags[i] == 1:
            primes.append(i)
    return primes

n = 2**21 - 1
%time print(f"{n} : {len(sieve(n))}")

2097151 : 155611
CPU times: user 456 ms, sys: 18.4 ms, total: 475 ms
Wall time: 479 ms


In [57]:
# sieve() 함수의 set 버전 - 차집합으로 구현
def set_sieve(n):
    primes = set(range(2, n+1))
    for i in range(2, int(n**0.5) + 1):
        if i in primes:
            primes -= set(range(i*i, n+1, i))
    return primes

n = 2**21 - 1
%time print(f"{n} : {len(set_sieve(n))}")

2097151 : 155611
CPU times: user 645 ms, sys: 149 ms, total: 794 ms
Wall time: 803 ms


- 에라토스테네스의 체 구현의 문제점
    - 메모리가 충분한가?
        - flags 배열을 저장할 수 있는 충분한 메모리 공간이 필요함
        - 메모리 공간이 충분히 확도 되었음에도 여전히 n이 너무 크다면?
            - 예) 2^1024 ...
        - 공간복잡도 측면에서 비효율적이다.

- 에라토스테네스의 체의 문제점 해결방안
    - 세그먼트(조각, 부분) 에라토스테네스의 체
        - n이 충분히 크면, n을 sqrt(n)개의 세그먼트로 분할
        - 첫 번째 세그먼트에서 소수를 모두 찾는다.
            - 이 소수들이 seed primes 가 된다.
        - seed primes로 다음 세그먼트들을 차례로 방문하며 처리한다.
        - 공간복잡도 문제는 O(n) -> O(sqrt(n))으로 더 효율적으로 보완이 되었지만 시간복잡도는 조금 희생된다.

In [54]:
def sieve(n):  # 에라토스테네스의 체
    flags = [0, 0] + [1] * (n-1)
    for i in range(2, int(n**0.5) + 1):
        if flags[i] == 1:
            for j in range(i*i, n+1, i):
                flags[j] = 0
    primes = []
    for i in range(len(flags)):
        if flags[i] == 1:
            primes.append(i)
    return primes


# 세그먼트 체(Segmented Sieve) 구현
def segmented_sieve(n):
    ## 1. 초기화 작업
    #-------------------------------------------------
    primes = []
    sqrtn = int(n**0.5)
    
    seeds = sieve(sqrtn)  # 시드(첫 번째 세그먼트의 소수들)
    
    low = sqrtn
    high = 2*sqrtn - 1
    # ------------------------------------------------
    
    ## 2. 두 번째 ~ 마지막 세그먼트를 반복해서 돌며 처리
    # ------------------------------------------------
    while low <= n:
        
        if high > n:  # 마지막 인덱스 보정(마지막 세그먼트의 high 값이 sqrt(n)이 아닐 시 임의로 지정함)
            high = n
            
        flags = [1] * sqrtn  # 실제 메모리에서 공간복잡도에 영향을 끼치는 부분
        
        ## 2-1. 각각의 세그먼트에서 시드의 배수 거르기
        for i in range(len(seeds)):
            start = low // seeds[i] * seeds[i]
            
            if start < low:  # 인덱스 보정
                start += seeds[i]
                
            for j in range(start, high+1, seeds[i]):
                flags[j - low] = 0
                
        ## 2-2. 각각의 세그먼트에서 판별 해놓은 소수만을 리스트(primes)에 추가하기
        for i in range(low, high+1):
            if flags[i - low] == 1:
                primes.append(i)
                
        low += sqrtn
        high += sqrtn
    # ------------------------------------------------
    return seeds + primes

n = 2**21 - 1
%time print(f"{n} : {len(segmented_sieve(n))}")

2097151 : 155611
CPU times: user 745 ms, sys: 15.7 ms, total: 761 ms
Wall time: 831 ms


#### 두 번째 소인수 분해 알고리즘

- 에라토스테네스의 체를 응용하여 구현

In [126]:
def add_factor(factors, n):
    if n not in factors:
        factors[n] = 1
    else:
        factors[n] += 1
    return factors


def get_MPF(n, mpf):
    if n in mpf:
        return mpf[n]
    else:
        for i in range(2, int(n**0.5) + 1):
            if n % i == 0:
                mpf[n] = i
                return i
        mpf[n] = n
        return n


def factorize_2(n):
    mpf = {}  # Minimum Prime Factor
    prime_factors = {}
    
    while n > 1:
        mpf_n = get_MPF(n, mpf)
        add_factor(prime_factors, mpf_n)
        n //= mpf_n
        
    return prime_factors

n = 360
print(f"{n} : {factorize_2(n)}")

360 : {2: 3, 3: 2, 5: 1}


- 메르센 소수(두 번째 소인수 분해 알고리즘으로 구현)

In [202]:
import time


num_of_prime = 0
start = time.time()
for i in range(2, 50):
    n = 2**i - 1
    factorization_2 = factorize_2(n)
    is_prime = "*" if len(factorization_2) == 1 else ""
    if is_prime:
        num_of_prime += 1
    # print(f"{i}, {n} : {factorization_2} {is_prime}")
end = time.time()
print(f"# of Prime : {num_of_prime}")
print(f"실행 시간 : {end - start:.4f}s")

# of Prime : 8
실행 시간 : 0.3104s


#### 쇼어 양자 알고리즘

- 기존 까지는 소인수 분해는 다항 시간에는 불가능이라 생각
    - 피터 쇼어: "양자 컴퓨터만 있으면 가능하다."
        - Shor, Peter W. "Algorithms for quantum computation: discrete logarithms and factoring." Proc. 35th ann. symp. on found. of comp. science. IEEE, 1994
- 두 개의 큰 소수로 구성된 합성수의 소인수 분해 문제
    - N = p x q 일 때, 매우 큰 소수 p, q에 대해서 소인수 분해가 가능한가?
    - 고전 컴퓨터로는 지수시간 복잡도를 극복할 수 없음
        - 현재 가장 효율적인 소인수 분해 알고리즘이라고 알려져 있는 GNFS(Ganeral Number Field Sieve)의 지수시간 복잡도는 O(e^1/3n)이다.
    - 두 소수의 합성수 N을 이용해서 p, q를 공개키, 비밀키로 사용 -> RSA 암호화
    - 기존 소인수 분해는 지수시간(2^b, b = 비트 수)을 극복하지 못 함.
        - 만약 다항시간에 소인수 분해가 가능하다면? -> RSA 붕괴 -> 지구 멸망?
- 피터 쇼어가 제안한 소인수 분해 알고리즘
    - 입력: 합성수 N(N = p x q 이고 p와 q는 소수)
    - 출력: N의 소인수 p, q
    1. 1보다 크고 N보다 작은 정수 a를 임의로 선택
    2. 만일, gcd(N, a) != 1이면, p를 찾은 것이므로, p = gcd(N, a), q = N / gcd(N, a)
    3. 함수 f(x) = a^x(mod N)의 주기 r을 찾는다.
        - 여기서 찾은 주기 r이 짝수가 아니면, 1번 단계부터 다시 시작한다.
    4. 주기 r로부터 두 개의 최대공약수 gcd_1, gcd_2를 찾는다.
        - gcd_1 = gcd(N, a^r/2 + 1), gcd_2 = gcd(N, a^r/2 - 1)
    5. gcd_1, gcd_2가 1, N이라면 1번 단계부터 다시 시작한다.
        - 아니면, 마침내 소인수들을 찾았으므로 p = gcd_1, q = gcd_2 를 리턴한다.
- 양자 컴퓨터와 쇼어의 양자 알고리즘
    - 쇼어의 알고리즘에서 여전히 지수 시간 복잡도를 가지는 부분
        - f(x) = a^x(mod N) 함수의 주기 찾기
    - 퀀텀 푸리에 변환(QFT: Quantum Fourier Transform)
        - 양자 컴퓨터로 푸리에 변환을 병렬적으로 수행(비트 수와 무관)
        - 역푸리에 변환(I-QFT: Inverse-QFT)으로 주기를 찾을 수 있다.
    - 쇼어 알고리즘을 수행하기 위한 양자 컴퓨터의 조건
        - N의 비트수(b, log2{N}) + I-QFT를 실행하기 위한 비트 수(2b) = 3b개의 큐비트가 필요
        - RSA-2048을 깨기 위해서는 6,144 큐비트가 필요함
        - 단, 큐비트 수를 줄이기 위한 연구도 진행중

In [204]:
import random
import math


def findPeriodByModPow(N, a):
    for x in range(1, N):
        if mod_pow(a, x, N) == 1:
            return x
    return -1


def mod_pow(a, x, N):
    y = 1
    while x > 0:
        if x & 1:
            y = (y * a) % N
        x = x >> 1
        a = a**2 % N
    return y


def factorize_3(N):  # 피터 쇼어의 양자 알고리즘
    while 1:
        a = random.randint(2, N-1)
        gcd = math.gcd(N, a)
        
        if gcd != 1:
            return gcd, N // gcd
        
        r = findPeriodByModPow(N, a)
        if r & 1:
            continue
        
        gcd_1 = math.gcd(N, a**(r//2) + 1)
        gcd_2 = math.gcd(N, a**(r//2) - 1)
        if gcd_1 == 1 or gcd_2 == 1:
            continue
        
        return gcd_1, gcd_2

In [205]:
import time


N = [3 * 5, 991 * 997, 8191 * 127]
for i in N:
    start = time.time()
    p, q = factorize_3(i)
    print(f"N = {p} x {q}")
    end = time.time()
    print(f"실행 시간 : {end - start}")

N = 3 x 5
실행 시간 : 0.0004420280456542969
N = 997 x 991
실행 시간 : 0.3494589328765869
N = 8191 x 127
실행 시간 : 0.04762697219848633
